# Récupération de tous les commentaires

**Définition** : On récupère tous les events qui ont le champs `feedbackType` définit


In [ ]:
import pandas as pd
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector
#from typing import List, Dict


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Récupération des logs pour le mois de février et pour plus tard

In [ ]:
es_connector = ElasticsearchConnector(env='monolog')

QUERY = {
    "query": {
        "bool": { 
          "must": [
             {
                  "exists": {
                    "field": "feedbackType"
                  }
            }
          ]
        }
      }
}

In [ ]:
logs = es_connector.execute_query(QUERY, "logs-new")

In [ ]:
grouped = (logs.groupby(['idVisit', 'url']) # on group ensemble les idvisit sur les m^me url
    .agg(
        feedbackTypes=('feedbackType', lambda x: list(x)),  # Agréger toutes les valeurs de feedbackType
        maxLastActionDateTime=('lastActionDateTime', 'max')  # Trouver la valeur maximale de lastActionDateTime
    )
    .reset_index())

In [ ]:
filtered_grouped = grouped[grouped['feedbackTypes'].apply(lambda x: any(v not in ["negative", "positive"] for v in x))]

# On sépare en 2 colonnes la colonne 'feedbackTypes' qui contient la liste des commenatires
filtered_grouped = filtered_grouped.copy()
filtered_grouped.loc[:, 'answer'] = filtered_grouped['feedbackTypes'].apply(
    lambda x: x[0] if x[0] in ["negative", "positive"] else ""
)
filtered_grouped.loc[:, 'feedback'] = filtered_grouped['feedbackTypes'].apply(
    lambda x: x[1:] if x[0] in ["negative", "positive"] and len(x) > 1 else x
)
filtered_grouped = filtered_grouped.drop(columns=['feedbackTypes', 'idVisit'])


In [ ]:
filtered_grouped = filtered_grouped.rename(columns={'maxLastActionDateTime': 'date'})
filtered_grouped = filtered_grouped.sort_values(by='date', ascending=True)

In [ ]:
filtered_grouped

In [ ]:
current_date = datetime.now().strftime("%d-%m-%Y")

# Définir le nom du fichier avec la date
file_name = f"commentaires-{current_date}.csv"

# Sauvegarder le DataFrame dans un fichier CSV avec le nom de fichier formaté
filtered_grouped.to_csv(file_name, sep=";", index=False)